In [71]:
import requests;
import psycopg2;
import json;

In [74]:
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="localhost",
    database="joostdelaere",
    user="joostdelaere",
    password=""
)

cursor = conn.cursor()

In [75]:
# Delete the table
# delete_user_table_query = "DROP TABLE IF EXISTS user_info"
# cursor.execute(delete_user_table_query)

# Create the table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS user_info (
                    epoch DECIMAL,
                    nft_id TEXT,
                    timestamp TIMESTAMP,
                    lsu_amount DECIMAL,
                    staking_rewards DECIMAL,
                    interest_earnings DECIMAL,
                    PRIMARY KEY (epoch, nft_id)
                )''')

# Delete the table
# delete_state_table_query = "DROP TABLE IF EXISTS state_info"
# cursor.execute(delete_state_table_query)

# Create the table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS state_info (
                    epoch DECIMAL,
                    interest_rate DECIMAL,
                    liquidity DECIMAL,
                    PRIMARY KEY (epoch)
                )''')

# # Delete the table
delete_test_table_query = "DROP TABLE IF EXISTS apy_data"
cursor.execute(delete_test_table_query)

# # Create the table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS apy_data (
                    epoch DECIMAL,
                    timestamp TIMESTAMP,
                    staking_rewards_apy DECIMAL,
                    interest_earnings_apy DECIMAL,
                    total_earnings_apy DECIMAL,
                    PRIMARY KEY (epoch)
                );''')

# Execute the stored procedure
cursor.execute("CALL insert_apy_data();")


# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [55]:
# Call Radix network gateway status
url = "https://rcnet.radixdlt.com/status/gateway-status"

# Make a POST request to the Gateway Status endpoint
response = requests.post(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Print the entire response for analysis
    print(data)
else:
    # Print an error message if the request was not successful
    print("Error:", response.status_code)


{'ledger_state': {'network': 'kisharnet', 'state_version': 21050598, 'proposer_round_timestamp': '2023-07-09T14:36:03.478Z', 'epoch': 12048, 'round': 1157}, 'release_info': {'release_version': '0.3.1-develop', 'open_api_schema_version': '0.3.0'}}


In [39]:
# Call Sundae component status to fetch: Hashmap, interest rate, liquidity, etc.
url = "https://rcnet.radixdlt.com/state/entity/details"

# Prepare the request body
request_body = {
    "addresses": [
        "component_tdx_c_1qved97g6ge57lkx95fza63hg2nn8aryjwj3tg596sgxs64u5sz"
    ],
    "aggregation_level": "global"
}

# Make a POST request to the State Entity Details endpoint
response = requests.post(url, json=request_body)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Extract proposer timestamp and epoch
    timestamp = data["ledger_state"]["proposer_round_timestamp"]
    
    epoch = data["ledger_state"]["epoch"]
    
    print(timestamp)
    print(epoch)
    
    # Extract the 'items' list from the 'data' variable
    items = data['items']

    # Extract the 'details' dictionary from each item
    details = items[0]['details']

    # Extract the 'state' dictionary from the 'details' dictionary
    state = details['state']

    # Extract the 'data_json' dictionary from the 'state' dictionary
    data_json = state['data_json']

    # Extract the 'fields' dictionary from the 'data_json' dictionary
    fields = data_json['fields']

    # Extract the 'map_entry' dictionary from the 'fields' dictionary
    map_entry = fields[6]

    # Access the 'entries' list from the map entry
    entries = map_entry['entries']    

    print(entries)

    # Iterate over the map entries and add them to the dictionary
    for entry in entries:
        nft_id = entry[0]
        lsu_amount = entry[1][1]
        staking_rewards = entry[1][2]
        interest_earnings = entry[1][3]
        
        # Insert the values into the table
        insert_query = """
        INSERT INTO user_info (epoch, nft_id, timestamp, lsu_amount, staking_rewards, interest_earnings)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (epoch, nft_id, timestamp, lsu_amount, staking_rewards, interest_earnings))
    
    # Commit the changes to the database
    conn.commit()

    # Close the database connection
    conn.close()

else:
    # Print an error message if the request was not successful
    print("Error:", response.status_code)


2023-07-09T12:58:16.434Z
12045
[['#1#', ['10899', '50', '33.333333333333333325', '0.625']], ['#2#', ['10899', '100', '66.66666666666666665', '1.25']], ['#3#', ['11098', '50', '0.000000000000000025', '0.625']]]


UniqueViolation: duplicate key value violates unique constraint "user_info_pkey"
DETAIL:  Key (epoch, nft_id)=(12045, #1#) already exists.
